In [1]:
import numpy as np
import os
import train
def load_mnist():
    data_dir = '../data'

    fd = open(os.path.join(data_dir, 'train-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trX = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trY = loaded[8:].reshape((60000)).astype(np.int)

    fd = open(os.path.join(data_dir, 't10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teX = loaded[16:].reshape((10000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 't10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teY = loaded[8:].reshape((10000)).astype(np.int)

    trY = np.asarray(trY)
    teY = np.asarray(teY)

    perm = np.random.permutation(trY.shape[0])
    trX = trX[perm]
    trY = trY[perm]

    perm = np.random.permutation(teY.shape[0])
    teX = teX[perm]
    teY = teY[perm]

    return trX, trY, teX, teY


def print_digit(digit_pixels, label='?'):
    for i in range(28):
        for j in range(28):
            if digit_pixels[i, j] > 128:
                print('#')
            else:
                print ('.')
        print ('')

    print ('Label: ', label)

print("executed")

executed


In [2]:
biases =[]
weights=[]
def cost_derivative(output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

class neural_net(object):
    def __init__(self, sizes):
        
        global biases
        global weights
        self.size = sizes
        for i in sizes[1:]:
            bias = np.random.randn(i,1)
            biases.append(bias)
        for i,j in zip(sizes[1:3],sizes[:2]):
            weight = np.random.randn(i,j)
            weights.append(weight)
        #print(weights)
    def feedforward(self,activation):
        global biases
        global weights
        zs=[]
        As=[]
        activation = activation.reshape(28,28)
        activation  = activation.reshape(784,1)
        for i in range(0,2):
            w=weights[i]
            b =  biases[i]
            z=np.dot(w,activation)+b
            #bigfloat.exp(5000,bigfloat.precision(100))
            activation = 1/(1+np.exp(-1*z))
            As.append(activation)
            zs.append(z)
        #print(np.argmax(As[-1]))
        return zs,As
        #print(As[-1],max(As[-1]),np.argmax(As[-1])) 
    def feedforwardtest(self,activation):
        global biases
        global weights
        zs=[]
        As=[]
        activation = activation.reshape(28,28)
        activation  = activation.reshape(784,1)
        for i in range(0,2):
            w=weights[i]
            b =  biases[i]
            z=np.dot(w,activation)+b
            #bigfloat.exp(5000,bigfloat.precision(100))
            activation = 1/(1+np.exp(-1*z))
            As.append(activation)
            zs.append(z)
        #print(np.argmax(As[-1]))
        return As[-1]
    def backprop(self,x,y):
        global biases
        global weights
        zs,As = self.feedforward(x)
        delta = cost_derivative(As[-1],y) * sigmoid_prime(zs[-1])
        update_bias = [np.zeros(b.shape) for b in  biases]
        update_weight = [np.zeros(w.shape) for w in weights]
        update_bias[-1] =delta
        update_weight[-1]  = np.dot(delta, As[-2].transpose())
        for layer in range(2,2):
            z = zs[-layer]
            sp = sigmoid_prime(z)
            delta = np.dot(weights[-layer+1].transpose(), delta) * sp
            update_bias[-layer] = delta
            update_weight[-layer] = np.dot(delta, As[-layer-1].transpose())
            #print("**********************")
            #print(update_bias)
        return (update_bias, update_weight)
    
    def create_mini_batches(self,training_data_x,training_data_y):
        eta = 3.0
        mini_batch_size = 10
        epochs = 30
        mini_batches_x =[]
        mini_batches_y =[]
        n = len(training_data_x)
        for j in range(epochs):
            for k in range(0,n,mini_batch_size):
                mini_batch_x = training_data_x[k:k+mini_batch_size]
                mini_batch_y = training_data_y[k:k+mini_batch_size]
                mini_batches_x.append(mini_batch_x)
                mini_batches_y.append(mini_batch_y)
            
            for mini_batch_x,mini_batch_y in zip(mini_batches_x,mini_batches_y):
                #print("next mini batch")
                self.update_mini_batch(mini_batch_x,mini_batch_y, eta)
   
    def update_mini_batch(self,mini_batch_x,mini_batch_y,eta):
        global biases
        global weights
        #newbias =[]
        #newweight=[]
        #print(biases[0].shape)
        #print(biases[1].shape)
        newbias = [np.zeros(b.shape) for b in  biases]
        newweight = [np.zeros(w.shape) for w in weights]
        '''for b,w in ( biases,weights):
            tempb = np.zeros(b.shape)
            tempw = np.zeros(w.shape)
            print("###")
            print(tempb.shape)
            newbias.append(tempb)
            newweight.append(tempw)'''

        for x,y in zip(mini_batch_x,mini_batch_y):
            delta_b ,delta_w = self.backprop(x,y)
            #print("#####")
            #print(delta_w[0].shape,delta_w[1].shape)
            #print(newweight[0].shape,newweight[1].shape)
            #print(delta_b[0].shape,delta_b[1].shape)
            i=0
            for del_b,b in zip(newbias,delta_b):
                newbias[i] = del_b + b
                i = i+1
            i=0
            for del_w,w in zip(newweight,delta_w):
                newweight[i] = del_w + w
                i=i+1
        new_final_weights=[]
        for each_w, del_w in zip(weights,newweight):
            weight1 = each_w - (eta/ len(mini_batch_x))*del_w
            #print(weight1.shape)
            #weight1[weight1>2.7] =2.7
            #weight1[weight1<(-2.7)] =-2.7
            new_final_weights.append(weight1)
        print(new_final_weights)
        weights = new_final_weights

        new_final_biases=[]
        for each_b, del_b in  zip(biases,newbias):
            bias1 = each_b - (eta/ len(mini_batch_x))*del_b
            new_final_biases.append(bias1)
        biases = new_final_biases

    def testFunction(self,testX,testY):
        test_results = [(np.argmax(self.feedforwardtest(x)), y)
                        for (x, y) in zip(testX,testY)]
        return sum(int(x == y) for (x, y) in test_results)
print("executed")

executed


In [42]:
trainX, trainY, testX, testY = load_mnist()
#print "Shapes: ", trainX.shape, trainY.shape, testX.shape, testY.shape
#print "\nDigit sample"
#print_digit(trainX[1], trainY[1])
net = neural_net([784, 10, 10])
'''for x,y in zip(trainX[1;200],trainY[1:200]):
    #print(x,y)
    x = x.reshape(28,28)
    x  = x.reshape(784,1)
    print(y)'''
    #print weights[2]
#print ( biases[1])
net.create_mini_batches(trainX[1:100],trainY[1:100])
#print weights[2]
#fp = open("biases.txt",'w')
#fp.write(biases)
#f = open("weights.txt",'w')
#f.write(weights)

print(net.testFunction(testX[1:100],testY[1:100]))


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:39: RuntimeWarning: overflow encountered in exp


98


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: overflow encountered in exp


In [13]:
print(net.testFunction(testX[1:5000],testY[1:5000]))

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:56: RuntimeWarning: overflow encountered in exp


854


In [ ]:
net = neural_net([784, 10, 10])

In [37]:

print(weights[0][12:16,12:16])

[array([[ 1.0142418 , -2.69242686,  0.05610535, ...,  0.65877382,
         0.87077277,  0.49158738],
       [-0.59343245,  0.57939284, -0.28119945, ...,  0.34303576,
        -0.77009764,  0.73797119],
       [-0.3900438 , -0.288673  , -1.33619303, ...,  0.77852933,
         0.12880325, -2.05081199],
       ..., 
       [ 0.27931462,  1.05078512,  1.79751655, ...,  1.28652074,
        -0.69917399,  0.38174565],
       [ 0.48411152,  1.69152611,  0.89745016, ...,  0.75379368,
        -0.46006362, -0.05631043],
       [ 1.00201866,  0.49405585,  0.06367002, ...,  0.73525837,
        -0.64207392,  0.2605051 ]]), array([[-0.59735705,  1.38928772,  0.04623828,  0.14039754, -0.54290045,
         1.80197633, -1.8507679 ,  0.26169715, -2.32391436, -0.13482018],
       [ 0.73122462, -1.44145217, -0.8504607 ,  0.1248716 , -1.2993191 ,
        -0.61585451, -0.95044165,  0.96509276, -0.80455871,  0.3970687 ],
       [-0.04935669,  0.71214397, -0.90715145,  1.58306359, -0.05812726,
        -0.080795

In [43]:
print(weights[0][12:16,12:16])

[]
